In [1]:
import os
from pytube import YouTube
import moviepy.editor as mp

In [2]:
LINK = 'https://www.youtube.com/watch?v=zjkBMFhNj_g&t=9s'
FILENAME = 'karpathy_llms.mp4'
OUTPUT_PATH = '../data/audios/'

In [27]:
def Download(link, output_path, filename):
    youtubeObject = YouTube(link)
    youtubeAudio = youtubeObject.streams.get_audio_only('mp4')
    title = youtubeObject.streams[0].title
    desc = youtubeObject.description
    youtubeCaptions = youtubeObject.captions.get_by_language_code('en')
    if youtubeCaptions is None:
        youtubeCaptions = youtubeObject.captions.get_by_language_code('a.en')
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    try:
        youtubeAudio.download(output_path, filename)
    except Exception as e:
        print(f"An error has occurred, {e}")
    captions_filename = '.'.join(filename.split('.')[:-1]) + '_captions.xml'
    with open(f'{output_path}/{captions_filename}', 'w') as f:
        f.write(youtubeCaptions.xml_captions)
    print("Download is completed successfully")
    desc_filename = '.'.join(filename.split('.')[:-1]) + '_desc.txt'
    with open(f'{output_path}/{desc_filename}', 'w') as f:
        f.write(f'{title}\n\n{desc}')
    # return title, desc

In [28]:
Download(LINK, OUTPUT_PATH, FILENAME)

Download is completed successfully


In [18]:
clip = mp.AudioFileClip(f'{OUTPUT_PATH}{FILENAME}').subclip(0, 300)
clip.write_audiofile(f"{OUTPUT_PATH}{'.'.join(FILENAME.split('.')[:-1])}_5min.wav")

MoviePy - Writing audio in ../data/audios/karpathy_llms_5min.wav


MoviePy - Done.


In [5]:
import whisperx
import ffmpeg

In [6]:
model = whisperx.load_model("tiny", 'cuda', compute_type='float16', language='en')

config.json:   0%|          | 0.00/2.25k [00:00<?, ?B/s]

vocabulary.txt:   0%|          | 0.00/460k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

model.bin:   0%|          | 0.00/75.5M [00:00<?, ?B/s]

Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.2.1. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint C:\Users\Andrii\.cache\torch\whisperx-vad-segmentation.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.2.1+cu118. Bad things might happen unless you revert torch to 1.x.


In [12]:
audio = whisperx.load_audio(f"{OUTPUT_PATH}{'.'.join(FILENAME.split('.')[:-1])}.wav")
result = model.transcribe(audio, batch_size=16)

In [13]:
result

{'segments': [{'text': " Hi everyone, so recently I gave a 30 minute talk on large language models, just kind of like an intro talk. Unfortunately that talk was not recorded, but a lot of people came to me after the talk and they told me that they really liked the talk. So I thought I was just re-recorded and basically put it up on YouTube. So here we go, the busy person's intro to large language models, director Scott. Okay, so let's begin. First of all, what is a large language model really? Well, a large language model is just two files, right?",
   'start': 0.009,
   'end': 29.36},
  {'text': " There will be two files in this hypothetical directory. So for example, we're going to do a specific example of the llama 270B model. This is a large language model released by meta AI. And this is basically the llama series of language models, the second iteration of it. And this is the 70 billion parameter model of",
   'start': 29.36,
   'end': 50.026},
  {'text': " of the series. So ther

In [14]:
import json

In [15]:
with open(f'{OUTPUT_PATH}/karpathy_llms_whisper_res.json', "w") as file:
    file_content = json.dumps(result, ensure_ascii=False)
    file_content = file_content.replace("{", "{\n")
    file_content = file_content.replace("}", "}\n")
    file_content = file_content.replace("\",", "\",\n")
    file.write(file_content)